## <font size=5> <strong>PHISHING URL WEBSITE DETECTION USING ML
 

## I. Importing essential libraries

In [1]:
## Import different libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from sklearn.metrics import confusion_matrix
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
import pickle
from sklearn.utils import resample


import warnings
warnings.filterwarnings('ignore')

## II. Importing and understanding our dataset 

In [2]:
## Reading the data
data = pd.read_csv('phishing.csv')

#### Verifying it as a 'dataframe' object in pandas

In [3]:
type(data)

pandas.core.frame.DataFrame

#### Shape of dataset

In [4]:
data.shape

(2456, 31)

#### Printing out a few columns

In [5]:
data.head(5)

,having_IP_Address,URL_Length,Shortining_Service,having_At_Symbol,double_slash_redirecting,Prefix_Suffix,having_Sub_Domain,SSLfinal_State,Domain_registeration_length,Favicon,...,popUpWidnow,Iframe,age_of_domain,DNSRecord,web_traffic,Page_Rank,Google_Index,Links_pointing_to_page,Statistical_report,Result
0,1,1,0,0,1,-1,-1,-1,0,0,...,0,0,-1,1,-1,-1,0,1,1,1
1,0,1,0,0,0,-1,0,1,0,0,...,0,0,-1,1,0,-1,0,1,0,1
2,0,0,0,0,0,-1,-1,-1,0,0,...,0,0,0,1,1,-1,0,0,1,1
3,0,0,0,0,0,-1,-1,-1,1,0,...,0,0,-1,1,1,-1,0,-1,0,1
4,0,0,1,0,0,-1,1,1,0,0,...,1,0,-1,1,0,-1,0,1,0,0


In [6]:
data.sample(5)

,having_IP_Address,URL_Length,Shortining_Service,having_At_Symbol,double_slash_redirecting,Prefix_Suffix,having_Sub_Domain,SSLfinal_State,Domain_registeration_length,Favicon,...,popUpWidnow,Iframe,age_of_domain,DNSRecord,web_traffic,Page_Rank,Google_Index,Links_pointing_to_page,Statistical_report,Result
1333,0,-1,0,0,0,0,1,1,1,0,...,0,0,0,0,1,1,1,0,0,0
399,0,-1,0,0,0,0,-1,-1,0,0,...,0,0,-1,1,-1,-1,0,1,0,1
1252,0,-1,0,0,0,0,0,1,0,0,...,0,0,-1,0,0,-1,0,0,0,0
1717,0,1,0,0,0,1,1,1,1,1,...,1,1,1,1,1,1,0,1,1,0
544,0,-1,0,0,0,-1,-1,1,0,0,...,0,0,1,0,1,1,1,0,0,0


#### Description

In [7]:
data.describe()

,having_IP_Address,URL_Length,Shortining_Service,having_At_Symbol,double_slash_redirecting,Prefix_Suffix,having_Sub_Domain,SSLfinal_State,Domain_registeration_length,Favicon,...,popUpWidnow,Iframe,age_of_domain,DNSRecord,web_traffic,Page_Rank,Google_Index,Links_pointing_to_page,Statistical_report,Result
count,2456.000000,2456.000000,2456.000000,2456.000000,2456.000000,2456.000000,2456.000000,2456.000000,2456.000000,2456.000000,...,2456.000000,2456.000000,2456.000000,2456.000000,2456.000000,2456.000000,2456.000000,2456.000000,2456.000000,2456.000000
mean,0.113192,-0.649837,0.122964,0.054560,0.125407,-0.254886,-0.185668,0.255700,0.018730,0.189739,...,0.196254,0.092020,-0.003257,0.536645,0.304560,-0.540717,0.139658,0.344463,0.179153,0.445440
std,0.316892,0.752690,0.328463,0.227166,0.331247,0.676170,0.802068,0.912332,0.798455,0.392175,...,0.397244,0.289112,0.939726,0.498757,0.834146,0.757832,0.346702,0.568911,0.383559,0.497115
min,0.000000,-1.000000,0.000000,0.000000,0.000000,-1.000000,-1.000000,-1.000000,-1.000000,0.000000,...,0.000000,0.000000,-1.000000,0.000000,-1.000000,-1.000000,0.000000,-1.000000,0.000000,0.000000
25%,0.000000,-1.000000,0.000000,0.000000,0.000000,-1.000000,-1.000000,-1.000000,-1.000000,0.000000,...,0.000000,0.000000,-1.000000,0.000000,0.000000,-1.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,-1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,1.000000,1.000000,-1.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,-1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,...,0.000000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000,1.000000,0.000000,1.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2456 entries, 0 to 2455
Data columns (total 31 columns):
 #   Column                       Non-Null Count  Dtype
---  ------                       --------------  -----
 0   having_IP_Address            2456 non-null   int64
 1   URL_Length                   2456 non-null   int64
 2   Shortining_Service           2456 non-null   int64
 3   having_At_Symbol             2456 non-null   int64
 4   double_slash_redirecting     2456 non-null   int64
 5   Prefix_Suffix                2456 non-null   int64
 6   having_Sub_Domain            2456 non-null   int64
 7   SSLfinal_State               2456 non-null   int64
 8   Domain_registeration_length  2456 non-null   int64
 9   Favicon                      2456 non-null   int64
 10  port                         2456 non-null   int64
 11  HTTPS_token                  2456 non-null   int64
 12  Request_URL                  2456 non-null   int64
 13  URL_of_Anchor                2456 non-null   int

#### columns:

In [9]:
data.columns

Index(['having_IP_Address', 'URL_Length', 'Shortining_Service',
       'having_At_Symbol', 'double_slash_redirecting', 'Prefix_Suffix',
       'having_Sub_Domain', 'SSLfinal_State', 'Domain_registeration_length',
       'Favicon', 'port', 'HTTPS_token', 'Request_URL', 'URL_of_Anchor',
       'Links_in_tags', 'SFH', 'Submitting_to_email', 'Abnormal_URL',
       'Redirect', 'on_mouseover', 'RightClick', 'popUpWidnow', 'Iframe',
       'age_of_domain', 'DNSRecord', 'web_traffic', 'Page_Rank',
       'Google_Index', 'Links_pointing_to_page', 'Statistical_report',
       'Result'],
      dtype='object')

In [10]:
l=data.columns

for i in data.columns:
    print(i)
    
    

having_IP_Address
URL_Length
Shortining_Service
having_At_Symbol
double_slash_redirecting
Prefix_Suffix
having_Sub_Domain
SSLfinal_State
Domain_registeration_length
Favicon
port
HTTPS_token
Request_URL
URL_of_Anchor
Links_in_tags
SFH
Submitting_to_email
Abnormal_URL
Redirect
on_mouseover
RightClick
popUpWidnow
Iframe
age_of_domain
DNSRecord
web_traffic
Page_Rank
Google_Index
Links_pointing_to_page
Statistical_report
Result


In [11]:
n = len(data[data['Result'] == 0])
print("The input URL is classified as legitimate (0)",n)

n1 = len(data[data['Result'] == 1])
print("The input URL is classified as phishing (1):",n1)



The input URL is classified as legitimate (0) 1362
The input URL is classified as phishing (1): 1094


#### Analysing the 'target' variable

In [12]:
data["Result"].describe()

count    2456.000000
mean        0.445440
std         0.497115
min         0.000000
25%         0.000000
50%         0.000000
75%         1.000000
max         1.000000
Name: Result, dtype: float64

In [13]:
data["Result"].unique()

array([1, 0], dtype=int64)

In [14]:
## Checking the missing values
data.isnull().sum()

having_IP_Address              0
URL_Length                     0
Shortining_Service             0
having_At_Symbol               0
double_slash_redirecting       0
Prefix_Suffix                  0
having_Sub_Domain              0
SSLfinal_State                 0
Domain_registeration_length    0
Favicon                        0
port                           0
HTTPS_token                    0
Request_URL                    0
URL_of_Anchor                  0
Links_in_tags                  0
SFH                            0
Submitting_to_email            0
Abnormal_URL                   0
Redirect                       0
on_mouseover                   0
RightClick                     0
popUpWidnow                    0
Iframe                         0
age_of_domain                  0
DNSRecord                      0
web_traffic                    0
Page_Rank                      0
Google_Index                   0
Links_pointing_to_page         0
Statistical_report             0
Result    

In [15]:
data.dtypes

having_IP_Address              int64
URL_Length                     int64
Shortining_Service             int64
having_At_Symbol               int64
double_slash_redirecting       int64
Prefix_Suffix                  int64
having_Sub_Domain              int64
SSLfinal_State                 int64
Domain_registeration_length    int64
Favicon                        int64
port                           int64
HTTPS_token                    int64
Request_URL                    int64
URL_of_Anchor                  int64
Links_in_tags                  int64
SFH                            int64
Submitting_to_email            int64
Abnormal_URL                   int64
Redirect                       int64
on_mouseover                   int64
RightClick                     int64
popUpWidnow                    int64
Iframe                         int64
age_of_domain                  int64
DNSRecord                      int64
web_traffic                    int64
Page_Rank                      int64
G

In [16]:
data.columns

Index(['having_IP_Address', 'URL_Length', 'Shortining_Service',
       'having_At_Symbol', 'double_slash_redirecting', 'Prefix_Suffix',
       'having_Sub_Domain', 'SSLfinal_State', 'Domain_registeration_length',
       'Favicon', 'port', 'HTTPS_token', 'Request_URL', 'URL_of_Anchor',
       'Links_in_tags', 'SFH', 'Submitting_to_email', 'Abnormal_URL',
       'Redirect', 'on_mouseover', 'RightClick', 'popUpWidnow', 'Iframe',
       'age_of_domain', 'DNSRecord', 'web_traffic', 'Page_Rank',
       'Google_Index', 'Links_pointing_to_page', 'Statistical_report',
       'Result'],
      dtype='object')

In [17]:
## Checking the missing values
final_df.isnull().sum()

NameError: name 'final_df' is not defined

## iii Exploratory Data Analysis (EDA)

In [ ]:
#Plotting the data distribution
data.hist(bins = 50,figsize = (15,15))
plt.show()

In [ ]:
sns.countplot(data['Result'])

## IV. Train Test split

In [ ]:
X=data.drop(['Result'],axis=1)
Y=data.Result

In [ ]:
! pip install imbalanced-learn

In [ ]:
# 42 input

from imblearn.over_sampling import SMOTE



rdsmple=SMOTE()
X_sampled,y_sampled=rdsmple.fit_resample(X,Y)

X_sampled.shape


X_sampled=pd.DataFrame(data=X_sampled,columns=X.columns)


X_sampled


sns.countplot(y_sampled)


X_sampled.columns


In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X_sampled,y_sampled,test_size=0.2,random_state=0)

In [ ]:
X_train.shape

In [ ]:
X_test.shape

In [ ]:
y_train.shape

In [ ]:
y_test.shape

## V. Model Fitting

In [ ]:
from sklearn.metrics import accuracy_score

### Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()

lr.fit(X_train,y_train)

Y_pred_lr = lr.predict(X_test)

In [ ]:
Y_pred_lr.shape

In [ ]:
score_lr = round(accuracy_score(Y_pred_lr,y_test)*100,2)

print("The accuracy score achieved using Logistic Regression is: "+str(score_lr)+" %")

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report 
cm = confusion_matrix(y_test, Y_pred_lr) 
print("Confusion Matrix is:\n",cm)


sns.set_style("dark")

sns.heatmap(cm, annot=True, cmap="Blues_r")
plt.title("Confusion Matrix for Logistic Regression", fontsize=14, fontname="Helvetica", y=1.04);

### SVM

In [ ]:
from sklearn import svm

sv = svm.SVC(kernel='rbf')

sv.fit(X_train, y_train)

Y_pred_svm = sv.predict(X_test)

In [ ]:
Y_pred_svm.shape

In [ ]:
score_svm = round(accuracy_score(Y_pred_svm,y_test)*100,2)

print("The accuracy score achieved using Linear SVM is: "+str(score_svm)+" %")

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report 
cm = confusion_matrix(y_test, Y_pred_svm) 
print("Confusion Matrix is:\n",cm)


sns.set_style("dark")

sns.heatmap(cm, annot=True, cmap="Blues_r")
plt.title("Confusion Matrix for Support vector machine", fontsize=14, fontname="Helvetica", y=1.04);


### K Nearest Neighbors

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=7)
knn.fit(X_train,y_train)
Y_pred_knn=knn.predict(X_test)

In [ ]:
Y_pred_knn.shape

In [ ]:
score_knn = round(accuracy_score(Y_pred_knn,y_test)*100,2)

print("The accuracy score achieved using KNN is: "+str(score_knn)+" %")

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report 
cm = confusion_matrix(y_test, Y_pred_knn) 
print("Confusion Matrix is:\n",cm)

sns.set_style("dark")

sns.heatmap(cm, annot=True, cmap="Blues_r")
plt.title("Confusion Matrix for KNN", fontsize=14, fontname="Helvetica", y=1.04);


### Decision Tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier



dt = DecisionTreeClassifier()
dt.fit(X_train,y_train)
Y_pred_dt = dt.predict(X_test)

In [ ]:
print(Y_pred_dt.shape)

In [ ]:
score_dt = round(accuracy_score(Y_pred_dt,y_test)*100,2)

print("The accuracy score achieved using Decision Tree is: "+str(score_dt)+" %")

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report 
cm = confusion_matrix(y_test, Y_pred_dt) 
print("Confusion Matrix is:\n",cm)

sns.set_style("dark")

sns.heatmap(cm, annot=True, cmap="Blues_r")
plt.title("Confusion Matrix for Decision Tree", fontsize=14, fontname="Helvetica", y=1.04);


### Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier


rf = RandomForestClassifier()
rf.fit(X_train,y_train)
Y_pred_rf = rf.predict(X_test)

In [ ]:
Y_pred_rf.shape

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report 
cm = confusion_matrix(y_test, Y_pred_rf) 
print("Confusion Matrix is:\n",cm)

sns.set_style("dark")

sns.heatmap(cm, annot=True, cmap="Blues_r")
plt.title("Confusion Matrix for Random Forest", fontsize=14, fontname="Helvetica", y=1.04);


In [ ]:
score_rf = round(accuracy_score(Y_pred_rf,y_test)*100,2)

print("The accuracy score achieved using Decision Tree is: "+str(score_rf)+" %")

## VI. Output final score

In [ ]:
scores = [score_lr,score_svm,score_knn,score_dt,score_rf]
algo = ["Logistic Regression","Support Vector Machine","K-Nearest Neighbors","Decision Tree","Random Forest"]    

for i in range(len(algo)):
    print("The accuracy score for "+algo[i]+" is: "+str(scores[i])+" %")

In [ ]:
sns.set(rc={'figure.figsize':(15,8)})
plt.xlabel("<------  Algorithms  ------>")
plt.ylabel("<------  Accuracy score  ------>")
plt.title("Comparsion of algorithms")

sns.barplot(algo,scores)

### classification report for each algorithm 

In [ ]:
l=[Y_pred_lr,Y_pred_svm,Y_pred_knn,Y_pred_dt,Y_pred_rf]
for i in range(len(algo)):
    print("-------------------------------------------------------")
    print(algo[i])
    print("-------------------------------------------------------")
    print("Classification Report is:\n\n",classification_report(y_test,l[i]))
    
    

In [ ]:
sns.set_style("whitegrid")
sns.color_palette("Paired")
plt.figure(figsize=(18,8), dpi=150)
ax = sns.barplot(x=algo, y=scores)
plt.title("Accuracy of Models")
plt.xlabel('Classification Models')
plt.ylabel("Accuracy of Classification Model")
for i in ax.patches:
    width, height = i.get_width(), i.get_height()
    x, y = i.get_xy() 
    ax.annotate(f'{round(height,2)}%', (x + width/2, y + height*1.02), ha='center')
plt.show()

# PREDICTION

In [ ]:
import pickle
filename = 'phishingurl.pkl'
pickle.dump(rf,open(filename,'wb'))

In [ ]:
model = open('phishingurl.pkl','rb')
forest = pickle.load(model)

In [ ]:
y_pred = forest.predict(X_test)

In [ ]:
X.columns

In [ ]:
n=list(map(int,input().split()))


In [ ]:
#1	1	0	0	1	-1	-1	-1	0	0	0	1	1	-1	1	-1	1	1	0	0	0	0	0	-1	1	-1	-1	0	1	1 phishing website
#0	0	1	0	0	-1	1	1	0	0	0	0	1	0	0	-1	0	0	0	1	0	1	0	-1	1	0	-1	0	1	0


In [ ]:
prediction=forest.predict([n])
if(prediction==0):
    print("it is a legitimate url")
else:
    print("it is a PHISING WEBSITE")